In [ ]:
import os
import requests

pdf_path="Human-Nutrition-text.pdf"

if not os.path.exists(pdf_path):
  print("The path does not contain any pdf")

  url="https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf"
  filename=pdf_path
  response=requests.get(url)

  if response.status_code==200:
    with open(filename,"wb") as file:
      file.write(response.content)
    print(f"The file has been downloaded and saved as {filename}")
  else:
    print(f"The file could not be downloaded. Status Code: {response.status_code}")

else:
  print(f"File {pdf_path} exists")

The path does not contain any pdf
The file has been downloaded and saved as Human-Nutrition-text.pdf


In [ ]:
pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 16.8 MB/s eta 0:00:00


In [ ]:
import fitz
from tqdm.auto import tqdm

def text_formatter(text):
  cleaned_text=text.replace("\n"," ").strip()
  #might need more preprocessing
  return cleaned_text

def open_and_read_pdf(pdf_path):
  doc=fitz.open(pdf_path)
  pages_and_texts=[]

  for page_number,page in tqdm(enumerate(doc)):
      text=page.get_text()
      text=text_formatter(text=text)
      pages_and_texts.append({
          "page_number":page_number-41,
          "page_char_count":len(text),
          "page_word_count":len(text.split(" ")),
          "page_sentence_count_raw":len(text.split(". ")),
          "page_token_count":len(text)/4,
          "text":text
      })
  return pages_and_texts


pages_and_texts=open_and_read_pdf(pdf_path=pdf_path)

0it [00:00, ?it/s]

In [ ]:
import random
random.sample(pages_and_texts,k=3)

[{'page_number': 919,
  'page_char_count': 1057,
  'page_word_count': 192,
  'page_sentence_count_raw': 9,
  'page_token_count': 264.25,
  'text': '“Man  wearing blue  shirt  standing on  white  surfboard”  by Alex  Blajan /  Unsplash  License  Older Adulthood: The  Golden Years  UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN  NUTRITION PROGRAM AND HUMAN NUTRITION PROGRAM  \xa0 The senior years are the period from age fifty-one until the end  of life. A number of physiological and emotional changes take place  during this life stage. For example, many older adults face serious  health challenges, such as cancer, heart disease, diabetes, or  dementia. Both men and women experience a loss of hormone  production, muscle mass, and strength and undergo changes in  body composition. Fat deposits build up in the abdominal area,  which increases the risk for Type 2 diabetes and cardiovascular  disease. The skin becomes thinner and may take longer to heal after  an injury. Around age sev

In [ ]:
import pandas as pd

df=pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,-41,29,4,1,7.25,Human Nutrition: 2020 Edition
1,-40,0,1,1,0.00,
2,-39,320,54,1,80.00,Human Nutrition: 2020 Edition UNIVERSITY OF ...
3,-38,212,32,1,53.00,Human Nutrition: 2020 Edition by University of...
4,-37,797,145,2,199.25,Contents Preface University of Hawai‘i at Mā...


In [ ]:
from spacy.lang.en import English
nlp=English()
nlp.add_pipe("sentencizer")

doc=nlp("This is a sentence. This another sentence.")
# assert len(list(doc.sents))==2

list(doc.sents)

[This is a sentence., This another sentence.]

In [ ]:
for item in tqdm(pages_and_texts):
  item["sentences"]=list(nlp(item["text"]).sents)

  item["sentences"]=[str(sentence) for sentence in item["sentences"]]

  item["page_sentence_count_spacy"] = len(item["sentences"])

  0%|          | 0/1208 [00:00<?, ?it/s]

In [ ]:
random.sample(pages_and_texts,k=1)

[{'page_number': 48,
  'page_char_count': 1680,
  'page_word_count': 309,
  'page_sentence_count_raw': 7,
  'page_token_count': 420.0,
  'text': 'that painlessly samples your blood, and downloads the information  to your cell phone, which has an app that evaluates the nutrient  profile of your blood and then recommends a snack or dinner menu  to assure you maintain adequate nutrient levels. What else is not  far off? How about another app that provides a shopping list that  adheres to all dietary guidelines and is emailed to the central server  at your local grocer, who then delivers the food to your home? The  food is then stored in your smart fridge which documents your  daily diet at home and delivers your weekly dietary assessment to  your home computer. At your computer, you can compare your diet  with other diets aimed at weight loss, optimal strength training,  reduction in risk for specific diseases or any other health goals you  may have. You also may delve into the field of n

In [ ]:
df=pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text,sentences,page_sentence_count_spacy
0,-41,29,4,1,7.25,Human Nutrition: 2020 Edition,[Human Nutrition: 2020 Edition],1
1,-40,0,1,1,0.00,,[],0
2,-39,320,54,1,80.00,Human Nutrition: 2020 Edition UNIVERSITY OF ...,[Human Nutrition: 2020 Edition UNIVERSITY OF...,1
3,-38,212,32,1,53.00,Human Nutrition: 2020 Edition by University of...,[Human Nutrition: 2020 Edition by University o...,1
4,-37,797,145,2,199.25,Contents Preface University of Hawai‘i at Mā...,[Contents Preface University of Hawai‘i at M...,2


In [ ]:
num_sentence_chunk_size = 10


def split_list(input_list,slice_size):

    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]


for item in tqdm(pages_and_texts):
    item["sentence_chunks"] = split_list(input_list=item["sentences"],
                                         slice_size=num_sentence_chunk_size)
    item["num_chunks"] = len(item["sentence_chunks"])

  0%|          | 0/1208 [00:00<?, ?it/s]

In [ ]:
random.sample(pages_and_texts, k=1)

[{'page_number': 289,
  'page_char_count': 52,
  'page_word_count': 13,
  'page_sentence_count_raw': 3,
  'page_token_count': 13.0,
  'text': 'PART\xa0V  CHAPTER 5. LIPIDS  Chapter 5. Lipids  |  289',
  'sentences': ['PART\xa0V  CHAPTER 5.',
   'LIPIDS  Chapter 5.',
   'Lipids  |  289'],
  'page_sentence_count_spacy': 3,
  'sentence_chunks': [['PART\xa0V  CHAPTER 5.',
    'LIPIDS  Chapter 5.',
    'Lipids  |  289']],
  'num_chunks': 1}]

In [ ]:
import re

# Split each chunk into its own item
pages_and_chunks = []
for item in tqdm(pages_and_texts):
    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]

        # Join the sentences together into a paragraph-like structure, aka a chunk (so they are a single string)
        joined_sentence_chunk = "".join(sentence_chunk).replace("  ", " ").strip()
        joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk) # ".A" -> ". A" for any full-stop/capital letter combo
        chunk_dict["sentence_chunk"] = joined_sentence_chunk

        # Get stats about the chunk
        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4 # 1 token = ~4 characters

        pages_and_chunks.append(chunk_dict)

# How many chunks do we have?
len(pages_and_chunks)

  0%|          | 0/1208 [00:00<?, ?it/s]

1843

In [ ]:
random.sample(pages_and_chunks, k=1)

[{'page_number': 955,
  'sentence_chunk': 'Image by Allison Calabrese / CC BY 4.0 of glucose is used as well. Glucose differs from fatty acids where glycogen storages can be depleted. As glycogen stores are depleted, fatigue will eventually set in. Figure 16.6 The Effect of Exercise Intensity on Fuel Sources The Fat-Burning Zone The fat-burning zone is a low intensity aerobic activity that keeps your heart rate between 60 and 69% of your maximum heart rate. The cardio zone on the other hand is a high intensity aerobic activity that keeps the heart rate between about 70 to 85% of your maximum heart rate. So which zone do you burn the most fat in?Technically, your body burns a higher percentage of of calories from fat during a low intensity aerobic activity but there’s more to it Fuel Sources | 955',
  'chunk_char_count': 764,
  'chunk_word_count': 135,
  'chunk_token_count': 191.0}]

In [ ]:
df = pd.DataFrame(pages_and_chunks)
df.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count
0,-41,Human Nutrition: 2020 Edition,29,4,7.25
1,-39,Human Nutrition: 2020 Edition UNIVERSITY OF HA...,308,42,77.00
2,-38,Human Nutrition: 2020 Edition by University of...,210,30,52.50
3,-37,Contents Preface University of Hawai‘i at Māno...,766,114,191.50
4,-36,Lifestyles and Nutrition University of Hawai‘i...,941,142,235.25


In [ ]:
# Show random chunks with under 30 tokens in length
min_token_length = 30
for row in df[df["chunk_token_count"] <= min_token_length].sample(5).iterrows():
    print(f'Chunk token count: {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_chunk"]}')

Chunk token count: 18.0 | Text: Updated July 24, 2017. Accessed April 15, 2018. 1112 | Threats to Health
Chunk token count: 25.0 | Text: Journal of the American Dietetic Association. 105(6), 929-36. Calories In Versus Calories Out | 1071
Chunk token count: 16.75 | Text: PART XI CHAPTER 11. TRACE MINERALS Chapter 11. Trace Minerals | 649
Chunk token count: 16.0 | Text: Accessed January 20, 2018. 1032 | The Effect of New Technologies
Chunk token count: 13.0 | Text: PART VII CHAPTER 7. ALCOHOL Chapter 7. Alcohol | 429


In [ ]:
pages_and_chunks_over_min_token_len = df[df["chunk_token_count"] > min_token_length].to_dict(orient="records")
pages_and_chunks_over_min_token_len[:2]

[{'page_number': -39,
  'sentence_chunk': 'Human Nutrition: 2020 Edition UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN NUTRITION PROGRAM ALAN TITCHENAL, SKYLAR HARA, NOEMI ARCEO CAACBAY, WILLIAM MEINKE-LAU, YA-YUN YANG, MARIE KAINOA FIALKOWSKI REVILLA, JENNIFER DRAPER, GEMADY LANGFELDER, CHERYL GIBBY, CHYNA NICOLE CHUN, AND ALLISON CALABRESE',
  'chunk_char_count': 308,
  'chunk_word_count': 42,
  'chunk_token_count': 77.0},
 {'page_number': -38,
  'sentence_chunk': 'Human Nutrition: 2020 Edition by University of Hawai‘i at Mānoa Food Science and Human Nutrition Program is licensed under a Creative Commons Attribution 4.0 International License, except where otherwise noted.',
  'chunk_char_count': 210,
  'chunk_word_count': 30,
  'chunk_token_count': 52.5}]

### Embedding Texts

In [ ]:
pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
embedding_model=SentenceTransformer(model_name_or_path="all-mpnet-base-v2",
                                    device="cpu")

sentences=[
    "You are my sunshine! My only sunshine!!!",
    "Modern Lonliness!! We are never alone but always depressed!!"
]

embeddings=embedding_model.encode(sentences)
embeddings_dict=dict(zip(sentences, embeddings))


for sentence,embedding in embeddings_dict.items():
  print("Sentence:",sentence)
  print("Embedding:",embedding)
  print("")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Sentence: You are my sunshine! My only sunshine!!!
Embedding: [ 2.53696889e-02  7.53752142e-02 -1.49296792e-02 -4.10102010e-02
  5.35754487e-02 -8.62715952e-03 -1.22529520e-02 -9.45850462e-03
 -1.54218255e-02  4.64595631e-02 -1.84940305e-02  3.21565978e-02
  4.85673500e-03 -7.82067925e-02  5.80261946e-02 -9.51470211e-02
  3.52578945e-02 -1.03772450e-02  3.93698364e-03  7.86765292e-03
  4.69562523e-02  2.60563213e-02  3.36934030e-02 -4.55385074e-03
 -3.80354598e-02  3.17195505e-02  3.94158512e-02 -2.27012262e-02
 -7.21307285e-03  1.98992882e-02 -6.89309984e-02  2.40316112e-02
 -3.85120101e-02 -4.23034988e-02  1.67251494e-06 -8.77147913e-03
  1.00425435e-02 -1.72678139e-02 -8.60042870e-03 -5.45682199e-03
 -4.46431432e-03 -4.15497199e-02 -1.12334527e-02 -1.56877004e-02
  1.53122433e-02  6.34042397e-02 -1.95131674e-02 -1.46307703e-02
  2.01473739e-02  1.72273163e-02 -1.80093688e-03  2.04830635e-02
 -4.68517793e-03 -1.54124498e-02  2.41920687e-02  1.51451053e-02
 -6.01190422e-03 -5.54860458

In [ ]:
text_chunks=[item["sentence_chunk"] for item in pages_and_chunks_over_min_token_len]

In [ ]:
embedding_model.to("cuda")
text_chunk_embeddings=embedding_model.encode(text_chunks,
                                             batch_size=32)#,
                                             #convert_to_tensor=True)
text_chunk_embeddings

array([[ 0.06742427,  0.09022819, -0.0050955 , ..., -0.02211547,
        -0.02321368,  0.01256907],
       [ 0.05521561,  0.05921394, -0.01661672, ..., -0.01204061,
        -0.01028473,  0.02273969],
       [ 0.02798017,  0.03398143, -0.02064266, ..., -0.00536188,
         0.02125599,  0.03130551],
       ...,
       [ 0.07705159,  0.00978557, -0.01218174, ..., -0.04086803,
        -0.0751763 , -0.02405259],
       [ 0.10304508, -0.01647023,  0.00826846, ..., -0.05742176,
        -0.02828035, -0.02946862],
       [ 0.08637746, -0.01253584, -0.01127464, ..., -0.05223799,
        -0.03367292, -0.02986602]], dtype=float32)

### Save Embeddings into a file

In [ ]:
i=0
for item in tqdm(pages_and_chunks_over_min_token_len):
  item["embedding"]=text_chunk_embeddings[i]
  i+=1

  0%|          | 0/1680 [00:00<?, ?it/s]

In [ ]:
pages_and_chunks_over_min_token_len[419]

{'page_number': 277,
 'sentence_chunk': 'often. • Calm your “sweet tooth” by eating fruits, such as berries or an apple. • Replace sugary soft drinks with seltzer water, tea, or a small amount of 100 percent fruit juice added to water or soda water. The Food Industry: Functional Attributes of Carbohydrates and the Use of Sugar Substitutes In the food industry, both fast-releasing and slow-releasing carbohydrates are utilized to give foods a wide spectrum of functional attributes, including increased sweetness, viscosity, bulk, coating ability, solubility, consistency, texture, body, and browning capacity. The differences in chemical structure between the different carbohydrates confer their varied functional uses in foods. Starches, gums, and pectins are used as thickening agents in making jam, cakes, cookies, noodles, canned products, imitation cheeses, and a variety of other foods. Molecular gastronomists use slow- releasing carbohydrates, such as alginate, to give shape and texture 

In [ ]:
text_chunks_and_embeddings_df=pd.DataFrame(pages_and_chunks_over_min_token_len)
embeddings_df_save_path="/content/text_chunks_and_embeddings_df.csv"
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path,index=False)

In [ ]:
text_chunks_and_embeddings_df_load=pd.read_csv(embeddings_df_save_path)
text_chunks_and_embeddings_df_load.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,-39,Human Nutrition: 2020 Edition UNIVERSITY OF HA...,308,42,77.00,[ 6.74242750e-02 9.02281851e-02 -5.09549817e-...
1,-38,Human Nutrition: 2020 Edition by University of...,210,30,52.50,[ 5.52156083e-02 5.92139438e-02 -1.66167226e-...
2,-37,Contents Preface University of Hawai‘i at Māno...,766,114,191.50,[ 2.79801711e-02 3.39814313e-02 -2.06426643e-...
3,-36,Lifestyles and Nutrition University of Hawai‘i...,941,142,235.25,[ 6.82566911e-02 3.81274782e-02 -8.46855436e-...
4,-35,The Cardiovascular System University of Hawai‘...,998,152,249.50,[ 3.30264196e-02 -8.49765539e-03 9.57158115e-...
